In [1]:
import pandas as pd

In [2]:
# Load the data from excel files into DataFrames

lhs_order_report = pd.read_excel("Company_X_Order_Report.xlsx")
lhs_pincode_zones = pd.read_excel("Company_X_Pincode_Zones.xlsx")
lhs_sku_master = pd.read_excel("Company_X_SKU_Master.xlsx")

rhs_invoice = pd.read_excel("Courier_Company_Invoice.xlsx")
rhs_rates = pd.read_excel("Courier_Company_Rates.xlsx")

In [3]:
# Merge data based on common columns

merged_data = lhs_order_report.merge(rhs_invoice, on='Order ID')
merged_data = merged_data.merge(lhs_pincode_zones, on='Customer Pincode')
merged_data = merged_data.merge(lhs_sku_master, on='SKU')

In [4]:
merged_data = merged_data.drop(['Warehouse Pincode_x','Warehouse Pincode_y'], axis=1)

In [5]:
merged_data.rename(columns={'Zone_x': 'Delivery Zone charged by Courier Company', 'Zone_y': 'Delivery Zone as per X'}, inplace=True)

In [6]:
# Calculate the total weight per order

merged_data["Total weight as per X (KG)"] = merged_data["Weight (g)"] * merged_data["Order Qty"] / 1000

In [7]:
def calculate_weight_slab_as_per_x(row):
    total_weight = row["Total weight as per X (KG)"]
    slabs_as_per_x = [0.25, 0.5, 0.75, 1.25, 1.5]
    
    for slab in slabs_as_per_x:
        if total_weight <= slab:
            return slab
    
    # If the total weight exceeds the highest slab, return the highest slab value
    return slabs_as_per_x[-1]

In [8]:
# Apply the function to calculate the weight slab as per X
merged_data["Weight slab as per X (KG)"] = merged_data.apply(calculate_weight_slab_as_per_x, axis=1)

In [9]:
weight_slabs_rates = {
    "a": {"slab_weight": 0.25, "fixed_charge": 29.5, "additional_weight_slab_charge": 23.6, "RTO Fixed Charge": 13.6, "RTO Additional Weight Slab Charge": 23.6},
    "b": {"slab_weight": 0.5, "fixed_charge": 33, "additional_weight_slab_charge": 28.3, "RTO Fixed Charge": 20.5, "RTO Additional Weight Slab Charge": 28.3},
    "c": {"slab_weight": 0.75, "fixed_charge": 40.1, "additional_weight_slab_charge": 38.9, "RTO Fixed Charge": 31.9, "RTO Additional Weight Slab Charge": 38.9},
    "d": {"slab_weight": 1.25, "fixed_charge": 45.4, "additional_weight_slab_charge": 44.8, "RTO Fixed Charge": 41.3, "RTO Additional Weight Slab Charge": 44.8},
    "e": {"slab_weight": 1.5, "fixed_charge": 56.6, "additional_weight_slab_charge": 55.5, "RTO Fixed Charge": 50.7, "RTO Additional Weight Slab Charge": 55.5}
}

In [10]:
# Create a function to calculate the total weight as per Courier Company (KG)
def calculate_total_weight_as_per_courier(row):
    zone = row["Delivery Zone charged by Courier Company"]
    charged_weight = row["Charged Weight"]
    
    # Retrieve the weight slab data for the zone
    slab_data = weight_slabs_rates.get(zone)
    
    if slab_data:
        slab_weight = slab_data["slab_weight"]
        
        # Calculate the total weight as per Courier Company (KG)
        total_weight_as_per_courier = (charged_weight // slab_weight) * slab_weight
        
        return total_weight_as_per_courier

In [11]:
# Apply the function to calculate the "Total weight as per Courier Company (KG)"
merged_data["Total weight as per Courier Company (KG)"] = merged_data.apply(calculate_total_weight_as_per_courier, axis=1)

In [12]:
def calculate_weight_slabs(row):
    weight = row["Charged Weight"]
    zone = row["Delivery Zone charged by Courier Company"]
    rate_row = rhs_rates[rhs_rates["Zone"] == zone].iloc[0]
    
    # Extract rate card data
    slab_weight = rate_row["Weight Slabs"]
    
    # Calculate the weight slab charged by Courier Company (KG)
    weight_slab_charged = weight - (weight // slab_weight) * slab_weight
    
    return weight_slab_charged

In [13]:
merged_data["Weight slab charged by Courier Company (KG)"] = merged_data.apply(calculate_weight_slabs, axis=1)

In [14]:
def calculate_expected_charge(row):
    weight = row["Total weight as per X (KG)"]
    zone = row["Delivery Zone as per X"]
    rate_row = rhs_rates[rhs_rates["Zone"] == zone].iloc[0]
    
    # Extract rate card data
    slab_weight = rate_row["Weight Slabs"]
    fwd_fixed = rate_row["Forward Fixed Charge"]
    fwd_additional = rate_row["Forward Additional Weight Slab Charge"]
    rto_fixed = rate_row["RTO Fixed Charge"]
    rto_additional = rate_row["RTO Additional Weight Slab Charge"]
    
    # Calculate slabs
    num_slabs = int(weight // slab_weight)
    total_charges = 0
    
    for _ in range(num_slabs):
        total_charges += fwd_additional
    total_charges += fwd_fixed
    
    if "rto" in row["Type of Shipment"].lower():
        for _ in range(num_slabs):
            total_charges += rto_additional
        total_charges += rto_fixed
    
    # Calculate COD charges
    if row["Payment Mode"] == "Prepaid":
        cod_charge = 0
    else:
        if row["Item Price(Per Qty.)"] * row["Order Qty"] <= 300:
            cod_charge = 15
        else:
            cod_charge = 0.05 * (row["Item Price(Per Qty.)"] * row["Order Qty"])
    
    # Calculate Total Charges
    total_charges += cod_charge
    
    return total_charges

In [15]:
merged_data["Expected Charge as per X (Rs.)"] = merged_data.apply(calculate_expected_charge, axis=1)

In [16]:
# Calculate the difference between expected and billed charges
merged_data["Difference (Rs.)"] = merged_data["Expected Charge as per X (Rs.)"] - merged_data["Billing Amount (Rs.)"]

In [17]:
merged_data.head()

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),AWB Code,Charged Weight,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Billing Amount (Rs.),Delivery Zone as per X,Weight (g),Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Expected Charge as per X (Rs.),Difference (Rs.)
0,2001827036,8904223818706,1,COD,233,1091122418320,1.6,173213,b,Forward charges,117.9,e,127,0.127,0.25,1.5,0.1,71.6,-46.3
1,2001821995,8904223818706,1,Prepaid,233,1091121183730,0.5,342008,d,Forward charges,45.4,b,127,0.127,0.25,0.0,0.5,33.0,-12.4
2,2001813009,8904223818706,1,COD,233,1091118553701,1.0,313001,d,Forward charges,92.2,b,127,0.127,0.25,0.0,1.0,48.0,-44.2
3,2001813009,8904223818706,1,COD,233,1091118553701,1.0,313001,d,Forward charges,92.2,b,127,0.127,0.25,0.0,1.0,48.0,-44.2
4,2001813009,8904223818706,1,COD,233,1091118553701,1.0,313001,d,Forward charges,92.2,b,127,0.127,0.25,0.0,1.0,48.0,-44.2


In [18]:
# Select and reorder columns for the output
resultant_data = merged_data[[
    "Order ID", "AWB Code", "Total weight as per X (KG)", "Weight slab as per X (KG)", 
    "Total weight as per Courier Company (KG)", "Weight slab charged by Courier Company (KG)",
    "Delivery Zone as per X", "Delivery Zone charged by Courier Company", "Expected Charge as per X (Rs.)",
    "Billing Amount (Rs.)", "Difference (Rs.)"
]]

In [24]:
# Save the resultant data to a CSV file
resultant_data.to_csv("Resultant_Data.csv", index=False)

In [32]:
# Calculate Total Charges for each order
merged_data["Total Charges"] = merged_data["Expected Charge as per X (Rs.)"] + merged_data["Billing Amount (Rs.)"]

# Calculate the number of orders and total amounts for each scenario
correctly_charged = merged_data[merged_data["Expected Charge as per X (Rs.)"] == merged_data["Billing Amount (Rs.)"]]
overcharged = merged_data[merged_data["Expected Charge as per X (Rs.)"] < merged_data["Billing Amount (Rs.)"]]
undercharged = merged_data[merged_data["Expected Charge as per X (Rs.)"] > merged_data["Billing Amount (Rs.)"]]

total_orders = len(merged_data)
total_correctly_charged = len(correctly_charged)
total_overcharged = len(overcharged)
total_undercharged = len(undercharged)

total_invoice_amount = merged_data["Billing Amount (Rs.)"].sum()
total_correctcharging_amount = correctly_charged["Billing Amount (Rs.)"].sum()
total_overcharging_amount = overcharged["Billing Amount (Rs.)"].sum()
total_undercharging_amount = undercharged["Billing Amount (Rs.)"].sum()

# Create a summary table
summary_table = pd.DataFrame({
    "Scenario": ["Correctly Charged", "Overcharged", "Undercharged", "Total"],
    "Count": [total_correctly_charged, total_overcharged, total_undercharged, total_orders],
    "Amount (Rs.)": [total_correctcharging_amount, total_overcharging_amount, total_undercharging_amount, total_invoice_amount]
})

# Save the summary table to an Excel file
with pd.ExcelWriter('result.xlsx', engine='xlsxwriter') as writer:
    merged_data.to_excel(writer, sheet_name='Resultant Data', index=False)
    summary_table.to_excel(writer, sheet_name='Summary Table', index=False)

print("Data saved to result.xlsx")

Data saved to result.xlsx
